在训练模型的过程中，数据处理和模型训练一定是分开并行进行的。特别是预训练的时候，数据处理负责读取预训练数据，分批次，并且tokenizer成向量。模型训练则只需要接受tokenizer之后的向量，不用去担心任何数据上的问题，这个在大规模预训练中，通常分为两个进程进行。

@QA \
@Q 为什么要这样分两个进程单独进行？ \
@A 1. 必须保证GPU资源不被浪费，GPU的任何等待数据的行为都是对GPU的极大浪费，所以必须保证GPU中每一个batch训练完成之后能够立马拿到下一个batch进行训练
2. 在大规模的预训练过程中，数据上成T规模的，只能一次读取一部分，边训练边处理数据。往往还需要使用集群进行处理。

# 词元化 / Tokenizer
Tokenizer的目的是将文本转换为模型可以处理的数据。因为模型只能够处理数字，因此，在让模型训练和推理之前，我们必须将文本转换成一段文本可以识别的数字序列。

@QA \
@Q Tokenizer的目标是什么？ \
TOkenizer的目标就是找到一种将原始文本转换为数字序列的方法，并且这种方法能够给出文本的最有意义的表示，能够很好的反映文本的语义信息，文本中词与词之间的关系，并且，这个表示要尽可能的小。

先用代码感受一下这个过程。下面使用transformers库的AutoTokenizer类，加载bert模型的tokenizer来，下面代码中的tokenize方法输出的是一个字符串列表，也就是token序列。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

## 输出
# ['Using', 'a', 'transform', '##er', 'network', 'is', 'simple']

这个过程也被称为tokenize，即将文本切分为一个个token的过程，直到获得可以用其词汇表表示的标记(token)。

接下来，将token转成输入ID序列。这个ID序列会用张量来表示，可以用作模型的输入。下面调用convert_tokens_to_ids方法来实现。


In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

# 输出
# [7993, 170, 11303, 1200, 2443, 1110, 3014]

模型的输出也是一个张量，张量中的每一个数字就对应了词表中的一个token，根据token，也就是组合成最终的文本输出。这就是解码decode过程。下面用代码来看一下。

In [ ]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)
# 输出
# 'Using a Transformer network is simple'

下面，我们来看几种tokenize方法。

# 基于词的分词方法 / Word based
基于词的分词方法就是按照单词进行拆分，如下图所示，将“Let's do tokenization!”拆分成“Let”、“'s”、“do”和“tokenization”这四个词。
<p align="cneter" width="80%">
    <img src="./imgs/tokenizer/word_base_tokenizer_example.png">
</p>
那么假如词表为：

In [ ]:
{
    "0": "I",
    "1": "am",
    "2": "a",
    "3": "let",
    "4": "'s'",
    "5": "see",
    "6": "do",
    "7": "tokenization"
}

那么经过tokenization之后，得到的张量为：[3,4,6,7]。

@Tips / 3 \
@1 如果tokenizer想要覆盖一种语言，那么就必须将每一个单词都加入词表中，并编码成一个token。英语中有超过50万个单词，那么词表就会非常大。而研究表明，最常用的5000个英语单词，在英文文本中的覆盖率为88.6%。所以理论上，只需要对这些代词进行编码，就足够了。\
@2 所以，在预训练一个大模型的过程中，往往是针对语料训练一个tokenizer。\
@3 除了文本中得到的token外，还有一些自定义token，比如“[UNK]”或"<unk>"，表示未登录词，也就是词表之外的词

@QA
@Q Word based方法的缺点？
@A Word based的tokenize方法中，会将“dog”，“dogs”视为不同的词，模型在最初无法知道这两个词是相似的。类似的还有“run”，“runing”。存在有大量的未登记词，可能会输出较多的“[UNK]。

# 基于字符的分词方法 / Character-based
就是按照“a”，“b”，“c”，“d”字符进行拆分，每个字符一个token，英语只有26个字符，所以加上标点和一些特殊字符的话，词表会小非常多，相应的未登记词也会少非常多。但是这个方法的缺点也很明显，单个字符本身包含的语义信息并不多，并且，一句话会被tokenize成非常长的token序列，对于模型来说，训练难度很大，还难有很好的效果。

# 子词分词法
所以就有了一种这中的方法，按照一个单词的子词进行拆分。英语中的单词，是有很多相同的子词的，比如“ing”，“ly”，“ang”等，子词法可以将“Let’s do tokenization!”切分为：
<p align="center" width="70%">
    <img src="./imgs/tokenizer/sub_word_base_tokenize.png">
</p>
这类方法包括 BPE 分词、WordPiece 分词和 Unigram 分词三种常见方法。下面简单讲一下BPE分词算法。

## BPE分词 
BPE 算法从一组基本符号（例如字母和边界字符）开始，迭代地寻找语料库中的两个相邻词元，并将它们替换为新的词元，这一过程被称为合并。合并的选择标准是计算两个连续词元的共现频率，也就是每次迭代中，最频繁出现的一对词元会被选择与合并。合并过程将一直持续达到预定义的词表大小。
<p align="center" width="70%">
    <img src="./imgs/tokenizer/bpe_case.png">
</p>


@Cite
Github RUCAIBox/LLMSurvey

@QA \
@Q 为了训练出一个高效的分词器，我们应该考虑哪些因素？
@A 1. 分词器必须具备无损重构的特性，能够将token准确无误的还原成输入文本。
2. 其次，分词器应具有高压缩率，即给定输入文本，经过分词处理后的词元数量应尽可能少，从而实现更为高效的文本编码和存储。压缩率计算公式为：
$ 压缩率 = \frac{UTF-8字节数}{词元数} $
例如，给定一段大小为 1MB（1,048,576 字节）的文本，如果它被分词为 200,000个词元，其压缩率即为 1,048,576/200,000=5.24。而目前主流的tokenizer对中文汉字的压缩率在1.5~1.6之间，也就是1.5~1.6个汉字划分为一个token。\
3. 预留token数量。除了从语料中学习到的token，还需要预留一些特殊的token，比如Qwen2.5的模型中，特殊token就有“<|endoftext|>”，“<|im_start|>”和“<|im_end|>”等。这些token都只作用与post-training阶段，表示一些特殊用于，比如任务隔离，角色隔离、function call等。

@Caution \
在选择LLM进行post-training时，还需要考虑开源LLM提供的分词器，是否能够满足需求。比如LLaMA模型的预训练语料以英语文本为主，所以训练的BPE分词器也是以英语为主的，在中文等其他语言时，表现可能不佳。所以，针对某些训练任务，我们可能需要拓展词表，在原有词表的基础上继续训练，也可以手动添加，比如添加某些自定义的token。

# 亲眼看看每个开源LLM的tokenizer / Tokenizer in opensource LLM
在huggingface上，每个开源模型的必须会开源tokenizer，文件就存在于我们下载的模型文件中，如下图所示：
<p align="center">
    <img src="./imgs/tokenizer/tokenizer_in_llm.png"  width="70%">
</p>
"vocab.json"里面存的就是词表，可以打开看一看。
tokenizer_config.json里面如下图所示：
<p align="center">
    <img src="./imgs/tokenizer/hf_tokenizer_case1.png" width="25%">
    <img src="./imgs/tokenizer/hf_tokenizer_case2.png" width="30%">
    <img src="./imgs/tokenizer/hf_tokenizer_case3.png" width="30%">
</p>

# 回到正题 / Back to Transformer
在本文的Transformer系列文章中，Tokenizer是怎么使用的。在本系列中，是用Transformer训练一个German-English的翻译任务，使用的数据集是Multi30k German-English数据集。使用的是spaCy这个库提供的英语和德语的tokenizer.


In [ ]:
import spacy
def load_tokenizers():
    try:
        spacy_de = spacy.load("de_core_news_sm")
    except IOError:
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    return spacy_de, spacy_en

下面是构建词表和加载词表的示例,有了上面的理论基础,下面的代码便能够很容易看懂了.

In [ ]:
def tokenize(text, tokenizer):
    return [tok.text for tok in tokenizer.tokenizer(text)]


def yield_tokens(data_iter, tokenizer, index):
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])



def build_vocabulary(spacy_de, spacy_en):
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    print("Building German Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_src = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_de, index=0),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    print("Building English Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_tgt = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_en, index=1),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_tgt.set_default_index(vocab_tgt["<unk>"])

    return vocab_src, vocab_tgt


def load_vocab(spacy_de, spacy_en):
    if not exists("vocab.pt"):
        vocab_src, vocab_tgt = build_vocabulary(spacy_de, spacy_en)
        torch.save((vocab_src, vocab_tgt), "vocab.pt")
    else:
        vocab_src, vocab_tgt = torch.load("vocab.pt")
    print("Finished.\nVocabulary sizes:")
    print(len(vocab_src))
    print(len(vocab_tgt))
    return vocab_src, vocab_tgt


if is_interactive_notebook():
    # global variables used later in the script
    spacy_de, spacy_en = show_example(load_tokenizers)
    vocab_src, vocab_tgt = show_example(load_vocab, args=[spacy_de, spacy_en])

@QA \
@Q Tokenizer和Embedding的关系是什么?
@A 1. Tokenizer是模型开始训练和推理的前置工作,而Embedding是模型训练和推理的第一步
2. Tokenizer将文本划分为一个个token,再将每个token映射成token_id,最终输出的是token向量. Embedding是将每个token编码成一个向量,假如输入的token长度为4096,每个token编码成512维的向量,那么Embedding输出为[4096, 512]维的矩阵.
3. Embedding可以很好的反映token之间的语义关系,利于模型的后续学习.